In [2]:
import torch

# Check available GPUs
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Number of available GPUs: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No GPUs available")

# Set the desired GPU
desired_gpu = 0  # Change this to the index of the GPU you want to use
if torch.cuda.is_available() and desired_gpu < torch.cuda.device_count():
    torch.cuda.set_device(desired_gpu)
    device = torch.device(f"cuda:{desired_gpu}")
    print(f"Using GPU {desired_gpu}: {torch.cuda.get_device_name(desired_gpu)}")
else:
    device = torch.device("cpu")
    print("Using CPU")

Number of available GPUs: 1
GPU 0: NVIDIA GeForce RTX 4070 Laptop GPU
Using GPU 0: NVIDIA GeForce RTX 4070 Laptop GPU


In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split

def train_resnet_classifier(data_dir, model_save_path, num_classes=2, num_epochs=25, batch_size=32, learning_rate=0.001, desired_gpu=0):
    # Set the desired GPU
    if torch.cuda.is_available():
        torch.cuda.set_device(desired_gpu)
        device = torch.device(f"cuda:{desired_gpu}")
    else:
        device = torch.device("cpu")

    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Load the dataset
    train_dir = os.path.join(data_dir, "train")
    val_dir = os.path.join(data_dir, "valid")
    test_dir = os.path.join(data_dir, "test")

    train_dataset = datasets.ImageFolder(train_dir, transform=transform)
    val_dataset = datasets.ImageFolder(val_dir, transform=transform)
    test_dataset = datasets.ImageFolder(test_dir, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Define the neural network
    class SimpleCNN(nn.Module):
        def __init__(self, num_classes):
            super(SimpleCNN, self).__init__()
            self.features = models.resnet18(pretrained=True)
            self.features.fc = nn.Linear(self.features.fc.in_features, num_classes)

        def forward(self, x):
            x = self.features(x)
            return x

    # Initialize the model, loss function, and optimizer
    model = SimpleCNN(num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    best_accuracy = 0.0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)

        # Validation loop
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), model_save_path)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")

    print("Training complete. Best accuracy: {:.4f}".format(best_accuracy))

# Example usage
train_resnet_classifier(data_dir="./datasets/Crowd Density", model_save_path="simple_cnn_model.pt")

c:\Users\timho\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\timho\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/25, Loss: 0.9720, Accuracy: 0.6333
Epoch 2/25, Loss: 0.3723, Accuracy: 0.9000
Epoch 3/25, Loss: 0.2160, Accuracy: 0.7500
Epoch 4/25, Loss: 0.1288, Accuracy: 0.8500
Epoch 5/25, Loss: 0.0528, Accuracy: 0.8167
Epoch 6/25, Loss: 0.0210, Accuracy: 0.6833
Epoch 7/25, Loss: 0.0076, Accuracy: 0.6333
Epoch 8/25, Loss: 0.0047, Accuracy: 0.8167
Epoch 9/25, Loss: 0.0029, Accuracy: 0.8667
Epoch 10/25, Loss: 0.0019, Accuracy: 0.8833
Epoch 11/25, Loss: 0.0014, Accuracy: 0.8833
Epoch 12/25, Loss: 0.0006, Accuracy: 0.8667
Epoch 13/25, Loss: 0.0006, Accuracy: 0.8667
Epoch 14/25, Loss: 0.0012, Accuracy: 0.8667
Epoch 15/25, Loss: 0.0005, Accuracy: 0.8667
Epoch 16/25, Loss: 0.0006, Accuracy: 0.8667
Epoch 17/25, Loss: 0.0006, Accuracy: 0.8500
Epoch 18/25, Loss: 0.0061, Accuracy: 0.8667
Epoch 19/25, Loss: 0.1370, Accuracy: 0.8167
Epoch 20/25, Loss: 0.2449, Accuracy: 0.8167
Epoch 21/25, Loss: 0.2666, Accuracy: 0.8167
Epoch 22/25, Loss: 0.1169, Accuracy: 0.7167
Epoch 23/25, Loss: 0.0558, Accuracy: 0.76

In [7]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


# Set the desired GPU
desired_gpu = 0  # Change this to the index of the GPU you want to use
if torch.cuda.is_available():
    torch.cuda.set_device(desired_gpu)
    device = torch.device(f"cuda:{desired_gpu}")
else:
    device = torch.device("cpu")

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset
data_dir = "./datasets/Crowd Density"
test_dir = os.path.join(data_dir, "test")

test_dataset = datasets.ImageFolder(test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Define the neural network
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = models.resnet18(pretrained=True)
        self.features.fc = nn.Linear(self.features.fc.in_features, num_classes)

    def forward(self, x):
        x = self.features(x)
        return x

# Load the best model
model = SimpleCNN(num_classes=2).to(device)
model.load_state_dict(torch.load("simple_cnn_model.pt", map_location=device))
model.eval()

# # Evaluate the model on the test dataset
# correct = 0
# total = 0
# with torch.no_grad():
#     for images, labels in test_loader:
#         images, labels = images.to(device), labels.to(device)
#         outputs = model(images)
#         _, predicted = torch.max(outputs, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# test_accuracy = correct / total
# print(f"Test Accuracy: {test_accuracy:.4f}")


# Evaluate the model on the test dataset
all_labels = []
all_predictions = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')
conf_matrix = confusion_matrix(all_labels, all_predictions)
class_report = classification_report(all_labels, all_predictions)

# Print metrics
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

c:\Users\timho\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\timho\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\timho\AppData\Local\Temp\ipykernel_34508\755879739.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/py

Test Accuracy: 0.9333
Test Precision: 0.9333
Test Recall: 0.9333
Test F1 Score: 0.9333
Confusion Matrix:
[[15  1]
 [ 1 13]]
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94        16
           1       0.93      0.93      0.93        14

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30

